In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import keras

Using TensorFlow backend.


In [2]:
labeled = pd.read_csv('train_bigger.csv', dtype={"from_2": object, "type": object, "category": object})

In [9]:
labeled.head()

,channel_type,channel_name,id,date,from,text,from_2,type,category
0,personal_chat,TZ155 Nontobeko Mthembu,226915,2018-03-22T17:59:29,Setup (Master),Sawubona Jessie 😊. ;;We can see that you are e...,setup,noncontent,setup
1,private_group,19:00 (10/07) Thabiso (2)🏆3️⃣,376217,2018-04-03T19:01:47,ThishaBot,Any questions?,bot,noncontent,conv
2,private_group,20:00 (11/07) Irfaan (2)🏆3️⃣,873757,2018-05-15T20:52:52,Irfaan Moolla,Today,student,noncontent,conv
3,private_group,13:00 (10/07) Fatima (3)🏆3️⃣,1368973,2018-07-02T19:05:13,TZ Simphiwe Mfaba,Kulungile 💪,tutor,noncontent,conv
4,private_group,20:00 (09/07) Jenna (1)🏆4⃣,1234119,2018-06-18T20:12:31,ThishaBot,"Surprisingly, the patient seems satisfied with...",bot,content,non mcq


In [10]:
labeled.dropna(subset=['category'], inplace=True)

In [11]:
unlabeled = pd.read_csv('labeled_output.csv')

In [12]:
unlabeled.dropna(subset=['text'], inplace=True)

In [13]:
unlabeled.drop(columns=['text_type'], axis=1, inplace=True)

In [16]:
train_texts = labeled['text'].values.tolist()
val_texts = unlabeled['text'].values.tolist()

In [17]:
from keras.preprocessing import sequence
from keras.preprocessing import text

TOP_K = 20000
MAX_SEQUENCE_LENGTH = 500

def sequence_vectorize(train_texts, val_texts):
    """Vectorizes texts as sequence vectors.

    1 text = 1 sequence vector with fixed length.

    # Arguments
        train_texts: list, training text strings.
        val_texts: list, validation text strings.

    # Returns
        x_train, x_val, word_index: vectorized training and validation
            texts and word index dictionary.
    """
    # Create vocabulary with training texts.
    tokenizer = text.Tokenizer(num_words=TOP_K)
    tokenizer.fit_on_texts(train_texts)

    # Vectorize training and validation texts.
    x_train = tokenizer.texts_to_sequences(train_texts)
    x_val = tokenizer.texts_to_sequences(val_texts)

    # Get max sequence length.
    max_length = len(max(x_train, key=len))
    if max_length > MAX_SEQUENCE_LENGTH:
        max_length = MAX_SEQUENCE_LENGTH

    # Fix sequence length to max value. Sequences shorter than the length are
    # padded in the beginning and sequences longer are truncated
    # at the beginning.
    x_train = sequence.pad_sequences(x_train, maxlen=max_length)
    x_val = sequence.pad_sequences(x_val, maxlen=max_length)
    return x_train, x_val, tokenizer.word_index